In [ ]:
import matplotlib as mpl
mpl.rcParams.update({'font.size': 12})
mpl.rcParams['axes.formatter.useoffset'] = False
import matplotlib.pyplot as plt
import numpy as np
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cmcrameri.cm as cm
import utils.load_grid as lg

from proteus.utils.constants import R_earth, M_earth

from proteus.plot.cpl_population import _get_mr_data
mrdata = _get_mr_data(os.environ.get("FWL_DATA"))

%matplotlib widget

: 

In [ ]:
pgrid_dir = "data/output/l9859b_grid3/"
cases = lg.get_cases(pgrid_dir)
ncases = len(cases)

In [ ]:
# Helpfiles
helps, hvars = lg.load_helpfiles(cases)

# Case numbers
cnums = ["%d"%int(s.split("_")[-1]) for s in cases]

# Statuses
statuses = lg.get_statuses(pgrid_dir)

print("Done")

In [ ]:
# observations (mass, radius) (value, +err, -err)

# obs = {
#     "Demangeon+21": [[1.94, 0.28, 0.28], [1.521, 0.119, 0.098]],
#     "Rajpaul+24":   [[2.14, 0.25, 0.29], []],
#     "Luque+22":     [[2.31, 0.46, 0.45], [1.58, 0.08, 0.08 ]],
#     "Cloutier+19":  [[2.31, 0.46, 0.45], [1.57, 0.14, 0.14]],
# }

obs = {
    "Demangeon+21": [[0.40, 0.16, 0.15], [0.85, 0.061, 0.047]],
    "Rajpaul+24":   [[0.47, 0.13, 0.15], []],
    "Cloutier+19":  [[1.01, 0.0,  1.01], [0.80, 0.05, 0.05 ]],
}

In [ ]:
plt.close("all")

In [ ]:
# parameters
z_key       = "Phi_global"
z_scale     = "linear"
z_max       = 1.0
z_min       = 0.1
jitter      = 0.025
plt_cnums   = False
only_done   = True
cmap        = cm.batlow
cmap.set_under('cyan')
cmap.set_over('magenta')

# Grid data
grd_x = lg.access_hvars(hvars,"M_tot",-1) / M_earth
grd_y = (lg.access_hvars(hvars,"z_obs",-1) + lg.access_hvars(hvars,"R_int",-1)) / R_earth
grd_z = lg.access_hvars(hvars,z_key,-1)

med_y = np.median(grd_y)
med_x = np.median(grd_x)

# Apply jitter to x
grd_x = np.random.random_sample(len(grd_x)) * jitter + grd_x - jitter/2

z_max = min(np.amax(grd_z), z_max)
z_min = max(np.amin(grd_z), z_min)
if z_scale == 'log':
    norm = mpl.colors.SymLogNorm(vmin=z_min, vmax=z_max,  linthresh=0.1)
else:
    norm = mpl.colors.Normalize(vmin=z_min, vmax=z_max)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

# make figure
fig,ax = plt.subplots(1,1, figsize=(8,6))
ax.autoscale(enable=True)
ax.set_xlabel(r"Mass [$M_\oplus$]")
ax.set_ylabel(r"Radius [$R_\oplus$]")

# plot grid
ax.scatter(grd_x, grd_y, c=grd_z, label="Simulations",
            cmap=cmap, norm=norm, zorder=6, s=30, marker='o')

# case numbers
if plt_cnums:
    for i in range(len(cnums)):
        x, y = grd_x[i], grd_y[i]
        txt = cnums[i]
        stat = statuses[int(cnums[i])]

        done = bool(10 <= stat <= 19)
        if done:
            c = 'k'
        else:
            c = 'r'
            if only_done:
                continue

        ax.text(x,y,txt,
                    fontsize=10, color=c,
                    verticalalignment='center', horizontalalignment='left', zorder=8)


# grid median
# ax.scatter(med_x, med_y, c='magenta', label="Median", s=300, marker='*', zorder=10)

# colorbar
cax = make_axes_locatable(ax).append_axes('right', size='5%', pad=0.05)
fig.colorbar(sm, cax=cax, orientation='vertical', extend='both').set_label(z_key)

# Plot observations
for k in obs.keys():
    _m = obs[k][0]
    _r = obs[k][1]

    if len(_m)+len(_r) == 6:
        ax.errorbar(_m[0],_r[0],
                    xerr=[[_m[2]],[_m[1]]], yerr=[[_r[2]],[_r[1]]],
                    marker='s', label=k, zorder=4)

    elif len(_m) == 3:
        c = ax.plot(_m[0],med_y)[0].get_color() # dummy colour
        ax.axvline(x=_m[0], label=k, c=c)

    elif len(_r) == 3:
        c = ax.plot(med_x,_r[0],)[0].get_color() # dummy colour
        ax.axhline(y=_r[0], label=k, c=c)

ax.set_xlim(right=1.1)
ax.set_ylim(bottom=0.6, top=1.6)

# Plot reference data
ax.autoscale(enable=False)
for k in mrdata.keys():
    if '1000 K' in k:
        continue
    ax.plot(mrdata[k][0],mrdata[k][1], label=k, linestyle='dashed',zorder=0)

ax.grid(zorder=0, alpha=0.2)
ax.legend(framealpha=1.0)
plt.show()
print(pgrid_dir)